# Systeme de RAG open source complete

In [9]:
"""
Système RAG 100% Open Source
Embeddings: sentence-transformers
VectorDB: ChromaDB
LLM: Mistral via Ollama (ou Groq)
"""

import os
from typing import List, Dict
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import requests
import json

class RAGSystem:
    def __init__(self, use_local_llm=True):
        """
        Initialise le système RAG
        
        Args:
            use_local_llm: True pour Ollama local, False pour API Groq
        """
        print("🚀 Initialisation du système RAG...")
        
        # 1. Modèle d'embeddings (multilingue, léger)
        print("📊 Chargement du modèle d'embeddings...")
        self.embedding_model = SentenceTransformer(
            'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
        )
        
        # 2. Base de données vectorielle ChromaDB
        print("💾 Initialisation de ChromaDB...")
        self.chroma_client = chromadb.Client(Settings(
            anonymized_telemetry=False,
            allow_reset=True
        ))
        
        # Créer ou récupérer une collection
        self.collection = self.chroma_client.get_or_create_collection(
            name="documents",
            metadata={"hnsw:space": "cosine"}  # Similarité cosinus
        )
        
        # 3. Configuration du LLM
        self.use_local_llm = use_local_llm
        if use_local_llm:
            self.llm_url = "http://localhost:11434/api/generate"
            self.llm_model = "mistral"
            print("🤖 LLM: Mistral (Ollama local)")
        else:
            # Pour utiliser Groq, définir GROQ_API_KEY dans .env
            self.groq_api_key = os.getenv("GROQ_API_KEY", "")
            self.llm_model = "llama3-8b-8192"
            print("🤖 LLM: Groq API")
        
        print("✅ Système RAG initialisé avec succès!\n")
    
    def add_documents(self, documents: List[str], metadatas: List[Dict] = None):
        """
        Ajoute des documents à la base vectorielle
        
        Args:
            documents: Liste de textes à indexer
            metadatas: Métadonnées optionnelles pour chaque document
        """
        print(f"📝 Ajout de {len(documents)} documents...")
        
        # Générer les embeddings
        embeddings = self.embedding_model.encode(
            documents,
            show_progress_bar=True,
            convert_to_numpy=True
        )
        
        # Créer des IDs uniques
        ids = [f"doc_{i}" for i in range(len(documents))]
        
        # Préparer les métadonnées si non fournies
        if metadatas is None:
            metadatas = [{"source": f"document_{i}"} for i in range(len(documents))]
        
        # Ajouter à ChromaDB
        self.collection.add(
            embeddings=embeddings.tolist(),
            documents=documents,
            metadatas=metadatas,
            ids=ids
        )
        
        print(f"✅ {len(documents)} documents ajoutés avec succès!\n")
    
    def search_similar(self, query: str, n_results: int = 3) -> Dict:
        """
        Recherche les documents les plus similaires à la requête
        
        Args:
            query: Question de l'utilisateur
            n_results: Nombre de documents à retourner
            
        Returns:
            Dictionnaire contenant documents, distances et métadonnées
        """
        # Générer l'embedding de la requête
        query_embedding = self.embedding_model.encode(
            [query],
            convert_to_numpy=True
        )
        
        # Recherche dans ChromaDB
        results = self.collection.query(
            query_embeddings=query_embedding.tolist(),
            n_results=n_results
        )
        
        return results
    
    def generate_answer_local(self, query: str, context: str) -> str:
        """
        Génère une réponse avec Ollama (local)
        """
        prompt = f"""Tu es un assistant IA qui répond aux questions en te basant sur le contexte fourni.

Contexte:
{context}

Question: {query}

Instructions:
- Réponds uniquement en te basant sur le contexte fourni
- Si la réponse n'est pas dans le contexte, dis-le clairement
- Sois précis et concis
- Cite les sources si possible

Réponse:"""

        payload = {
            "model": self.llm_model,
            "prompt": prompt,
            "stream": False,
            "options": {
                "temperature": 0.3,
                "num_predict": 512
            }
        }
        
        try:
            response = requests.post(self.llm_url, json=payload, timeout=60)
            response.raise_for_status()
            return response.json()["response"]
        except Exception as e:
            return f"❌ Erreur lors de la génération: {str(e)}"
    
    def generate_answer_groq(self, query: str, context: str) -> str:
        """
        Génère une réponse avec Groq API
        """
        if not self.groq_api_key:
            return "❌ GROQ_API_KEY non définie. Créez un fichier .env avec votre clé API."
        
        url = "https://api.groq.com/openai/v1/chat/completions"
        headers = {
            "Authorization": f"Bearer {self.groq_api_key}",
            "Content-Type": "application/json"
        }
        
        payload = {
            "model": self.llm_model,
            "messages": [
                {
                    "role": "system",
                    "content": "Tu es un assistant qui répond aux questions en te basant uniquement sur le contexte fourni."
                },
                {
                    "role": "user",
                    "content": f"Contexte:\n{context}\n\nQuestion: {query}"
                }
            ],
            "temperature": 0.3,
            "max_tokens": 512
        }
        
        try:
            response = requests.post(url, headers=headers, json=payload, timeout=30)
            response.raise_for_status()
            return response.json()["choices"][0]["message"]["content"]
        except Exception as e:
            return f"❌ Erreur Groq: {str(e)}"
    
    def query(self, question: str, n_results: int = 3) -> Dict:
        """
        Pipeline RAG complet: Question → Recherche → Génération
        
        Args:
            question: Question de l'utilisateur
            n_results: Nombre de documents à utiliser comme contexte
            
        Returns:
            Dictionnaire avec la réponse et les sources
        """
        print(f"❓ Question: {question}\n")
        
        # 1. Recherche vectorielle
        print("🔍 Recherche des documents pertinents...")
        search_results = self.search_similar(question, n_results)
        
        if not search_results['documents'][0]:
            return {
                "answer": "Aucun document pertinent trouvé dans la base.",
                "sources": []
            }
        
        # 2. Préparer le contexte
        documents = search_results['documents'][0]
        metadatas = search_results['metadatas'][0]
        distances = search_results['distances'][0]
        
        context = "\n\n---\n\n".join([
            f"[Document {i+1}] (Similarité: {1-dist:.2%})\n{doc}"
            for i, (doc, dist) in enumerate(zip(documents, distances))
        ])
        
        print(f"📚 {len(documents)} documents trouvés\n")
        
        # 3. Génération de la réponse
        print("🤖 Génération de la réponse...")
        if self.use_local_llm:
            answer = self.generate_answer_local(question, context)
        else:
            answer = self.generate_answer_groq(question, context)
        
        # 4. Préparer les sources
        sources = [
            {
                "content": doc[:200] + "..." if len(doc) > 200 else doc,
                "metadata": meta,
                "similarity": f"{(1-dist)*100:.1f}%"
            }
            for doc, meta, dist in zip(documents, metadatas, distances)
        ]
        
        return {
            "answer": answer,
            "sources": sources
        }
    
    def reset_database(self):
        """Réinitialise la base de données vectorielle"""
        self.chroma_client.reset()
        self.collection = self.chroma_client.get_or_create_collection(
            name="documents",
            metadata={"hnsw:space": "cosine"}
        )
        print("🗑️ Base de données réinitialisée")


# ============================================================================
# EXEMPLE D'UTILISATION
# ============================================================================

if __name__ == "__main__":
    # Initialiser le système RAG
    # use_local_llm=True pour Ollama, False pour Groq
    rag = RAGSystem(use_local_llm=True)
    
    # Documents exemples (remplacez par vos propres données)
    documents = [
        """L'intelligence artificielle (IA) est l'ensemble des théories et des techniques 
        mises en œuvre en vue de réaliser des machines capables de simuler l'intelligence humaine. 
        Elle regroupe des domaines comme l'apprentissage automatique, le traitement du langage 
        naturel et la vision par ordinateur.""",
        
        """Le RAG (Retrieval-Augmented Generation) est une technique qui combine la recherche 
        d'informations et la génération de texte. Il permet aux modèles de langage d'accéder 
        à des bases de connaissances externes pour fournir des réponses plus précises et à jour.""",
        
        """ChromaDB est une base de données vectorielle open source optimisée pour les embeddings. 
        Elle permet de stocker et rechercher des vecteurs de haute dimension de manière efficace, 
        ce qui la rend idéale pour les applications d'IA comme le RAG.""",
        
        """Mistral est un modèle de langage open source développé par Mistral AI en France. 
        Il offre d'excellentes performances en français et peut être exécuté localement via 
        des outils comme Ollama, sans nécessiter de connexion internet."""
    ]
    
    metadatas = [
        {"source": "wikipedia", "topic": "IA"},
        {"source": "documentation", "topic": "RAG"},
        {"source": "documentation", "topic": "ChromaDB"},
        {"source": "blog", "topic": "Mistral"}
    ]
    
    # Ajouter les documents
    rag.add_documents(documents, metadatas)
    
    # Poser des questions
    print("="*60)
    print("DÉMONSTRATION DU SYSTÈME RAG")
    print("="*60 + "\n")
    
    questions = [
        "Qu'est-ce que le RAG ?",
        "Pourquoi utiliser ChromaDB ?",
        "Quelles sont les capacités de Mistral ?"
    ]
    
    for i, question in enumerate(questions, 1):
        print(f"\n{'='*60}")
        print(f"QUESTION {i}")
        print('='*60)
        
        result = rag.query(question, n_results=2)
        
        print(f"\n💡 RÉPONSE:\n{result['answer']}\n")
        
        print("📖 SOURCES UTILISÉES:")
        for j, source in enumerate(result['sources'], 1):
            print(f"\n  [{j}] Similarité: {source['similarity']}")
            print(f"      Sujet: {source['metadata']['topic']}")
            print(f"      Extrait: {source['content'][:100]}...")
        
        print("\n" + "="*60)

🚀 Initialisation du système RAG...
📊 Chargement du modèle d'embeddings...
💾 Initialisation de ChromaDB...
🤖 LLM: Mistral (Ollama local)
✅ Système RAG initialisé avec succès!

📝 Ajout de 4 documents...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ 4 documents ajoutés avec succès!

DÉMONSTRATION DU SYSTÈME RAG


QUESTION 1
❓ Question: Qu'est-ce que le RAG ?

🔍 Recherche des documents pertinents...
📚 2 documents trouvés

🤖 Génération de la réponse...

💡 RÉPONSE:
❌ Erreur lors de la génération: 404 Client Error: Not Found for url: http://localhost:11434/api/generate

📖 SOURCES UTILISÉES:

  [1] Similarité: 45.6%
      Sujet: RAG
      Extrait: Le RAG (Retrieval-Augmented Generation) est une technique qui combine la recherche 
        d'inform...

  [2] Similarité: 19.5%
      Sujet: ChromaDB
      Extrait: ChromaDB est une base de données vectorielle open source optimisée pour les embeddings. 
        Ell...


QUESTION 2
❓ Question: Pourquoi utiliser ChromaDB ?

🔍 Recherche des documents pertinents...
📚 2 documents trouvés

🤖 Génération de la réponse...

💡 RÉPONSE:
❌ Erreur lors de la génération: 404 Client Error: Not Found for url: http://localhost:11434/api/generate

📖 SOURCES UTILISÉES:

  [1] Similarité: 63.6%
      Sujet: Ch